### Basics

In [1]:
!pip install groq

In [ ]:
from groq import Groq
from termcolor import colored

In [ ]:
api_key = "" # Your API key here

In [4]:
import builtins

def wrap_text(text, max_length=150):
    lines = text.splitlines(keepends=True)
    wrapped_lines = []

    for line in lines:
        if line.strip() == "":
            wrapped_lines.append("")
            continue

        words = line.split()
        current_line = []

        for word in words:
            if len(' '.join(current_line + [word])) > max_length:
                wrapped_lines.append(' '.join(current_line))
                current_line = [word]
            else:
                current_line.append(word)

        if current_line:
            wrapped_lines.append(' '.join(current_line))

        if line.endswith("\n"):
            wrapped_lines[-1] += "\n"

    return '\n'.join(wrapped_lines)

def print(*args, max_length=150, **kwargs):
    text = ' '.join(str(arg) for arg in args)
    wrapped_text = wrap_text(text, max_length=max_length)
    builtins.print(wrapped_text, **kwargs)


In [5]:
client = Groq(
    api_key=api_key
)

def generate_response(messages, model="llama-3.1-8b-instant", max_tokens=150, temperature=0.7):
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        max_tokens=max_tokens,
        temperature=temperature
    )

    response = chat_completion.choices[0].message.content
    return response

In [6]:
for i in range(2):
    user_message = input("Enter a prompt")

    print(colored("You:", "red"), user_message)

    if user_message == "exit":
        break

    response = generate_response([{"role": "user", "content": user_message}])

    print(colored("AI:", "green"), response)

You: I'm keller
AI: Hi Keller, it's nice to meet you. Is there something I can help you with or would you like to chat?
You: what's my name
AI: I don't have any information about your name. Our conversation just started, and I don't retain any personal data. If you'd like to share
your name, I'm happy to learn it and use it in our conversation.


### Memory

In [ ]:
def add_user_message(messages, user_message):
    messages.append({"role": "user", "content": user_message})
    return messages

def add_ai_message(messages, ai_message):
    messages.append({"role": "assistant", "content": ai_message})
    return messages

def add_system_message(messages, system_message):
    messages.append({"role": "system", "content": system_message})
    return messages 

def chat_with_ai(chat, user_message):
    chat = add_user_message(chat, user_message)
    print(colored("User:", "red"), user_message)
    response = generate_response(chat)
    chat = add_ai_message(chat, response)
    print(colored("AI:", "green"), response)
    return chat

In [8]:
chat = []
chat = add_system_message(chat, "You are a helpful assistant.")

while True:
    user_message = input("Enter a prompt: ")

    if user_message == "exit":
        break

    chat = chat_with_ai(chat, user_message)

print("\n\n\n---\n\n\nChat history:")
print(chat)

User: I'm keller
AI: Hello Keller, how are you doing today? Is there anything I can help you with or would you like to chat about something in particular?
User: what's my name?
AI: Your name is Keller.



---



Chat history:
[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': "I'm keller"}, {'role': 'assistant', 'content': 'Hello
Keller, how are you doing today? Is there anything I can help you with or would you like to chat about something in particular?'}, {'role': 'user',
'content': "what's my name?"}, {'role': 'assistant', 'content': 'Your name is Keller.'}]


*Note: We're pretending characters and tokens are the same in this example for the sake of simplicity.*

*In reality it is different based on the preprocessing. You cas see more here https://platform.openai.com/tokenizer.*

In [9]:
# Sliding Window Function
def maintain_window(chat, max_tokens=200):
    current_token_count = sum(len(msg['content']) for msg in chat)
    print("Current token count:", current_token_count)
    while current_token_count > max_tokens and len(chat) > 1:
        removed = chat.pop(1)  # Remove the oldest message
        current_token_count -= len(removed['content'])
    print("Token count:", current_token_count)
    return chat

# Example chat
chat = []
chat = add_system_message(chat, "You are a helpful assistant.")

chat = chat_with_ai(chat, "My name is Joe.")

chat = chat_with_ai(chat, "I need help planning a trip.")

chat = chat_with_ai(chat, "Can you recommend some activities for a weekend getaway?")

# Trim context if the token count exceeds the limit
chat = maintain_window(chat, max_tokens=200)

chat = chat_with_ai(chat, "Do you remember my name?")

print("\n\n\n---\n\n\nChat history:")
print(chat)

User: My name is Joe.
AI: Nice to meet you, Joe. Is there something I can help you with or would you like to chat?
User: I need help planning a trip.
AI: Planning a trip can be a bit overwhelming, but I'm here to help.


To get started, can you please tell me a bit more about your trip? Here are some questions to help me narrow down some options:


1. Where are you thinking of going?

2. How long do you have for the trip?

3. Are you looking for a relaxing vacation, an adventure, or something in between?

4. Are there any specific activities or experiences you want to have on your trip (e.g. hiking, sightseeing, trying new foods, visiting historical
sites)?

5. What is your budget for the trip?

6. Will you be traveling solo, with friends, or with family?


Let me know your answers to these questions and I
User: Can you recommend some activities for a weekend getaway?
AI: A weekend getaway sounds like a great idea, Joe. Here are some activity ideas for a short trip:


**Outdoor Activit

In [10]:
# Summarizing Function
def summarize_context(chat):
    summary_prompt = "Summarize the following conversation concisely:\n" + "\n".join(
        f"{msg['role']}: {msg['content']}" for msg in chat
    )
    summary = generate_response([{"role": "user", "content": summary_prompt}])
    print(colored("System:", "blue"), "Summary of prior conversation:", summary)
    return [{"role": "system", "content": "Summary of prior conversation: " + summary}]

# Example chat
chat = []
chat = add_system_message(chat, "You are a helpful assistant.")

chat = chat_with_ai(chat, "My name is Joe.")

chat = chat_with_ai(chat, "I need help planning a trip.")

chat = chat_with_ai(chat, "Can you recommend some activities for a weekend getaway?")

# Summarize context after multiple messages
chat = summarize_context(chat)

chat = chat_with_ai(chat, "Do you remember my name?")

print("\n\n\n---\n\n\nChat history:")
print(chat)

User: My name is Joe.
AI: Hi Joe, it's nice to meet you. I'm here to assist you with any questions, tasks, or just to chat if you'd like. What brings you here
today?
User: I need help planning a trip.
AI: Planning a trip can be a bit overwhelming, but I'm here to help. Can you give me a bit more information about what you're looking for?
For example:


- Where are you thinking of traveling to?

- What type of trip is it (beach, city, nature, adventure, etc.)?

- How long do you have for the trip?

- Are you traveling solo, with friends, or with family?

- What's your budget for the trip?


The more details you provide, the better I can assist you in planning your trip.
User: Can you recommend some activities for a weekend getaway?
AI: A weekend getaway sounds like a great idea, Joe. There are countless activities you can do depending on your interests and the location
you choose. Here are some ideas to get you started:


**Outdoor Activities**


1. Hiking or backpacking: Explore nearby

### Single Shot
- What are the limitations?
    - Can't correct it part way through, no opportunity for feedback or guidance.
    - Can't adjust the temperature. For idea gen we might want high temperature. For summarizing something lower.
    - Can't adjust the output length or where our tokens are spent.
    - Can't adjust the model used. We may want a smarter model for certain steps.

In [11]:
# Initialize the conversation with a system message
chat = []
chat = add_system_message(chat, "You are a creative and insightful storyteller.")

# Define the one-shot prompt
one_shot_prompt = """
You are a creative and insightful storyteller. Your task is to create a compelling story by brainstorming, exploring, outlining, and summarizing the narrative in a structured and concise way. Follow these steps in your response:

1. **Brainstorm**: Generate three unique story ideas in different genres. Each idea should be summarized in one sentence.

2. **Explore**: Pick the most interesting of the three ideas and expand on it. Provide a brief description of the protagonist, the main plot points, and any key themes or conflicts.

3. **Outline**: Create a structured outline for the chosen story. Include the main elements (beginning, middle, and end) and at least three key events or twists.

4. **Summarize**: Write a concise and engaging paragraph summarizing the story. This should read like a pitch or back-cover description that hooks the reader.

Be creative, thoughtful, and concise, ensuring your response covers each step clearly.
"""

# Add the user message with the one-shot prompt
chat = add_user_message(chat, one_shot_prompt)

# Generate the response using your function
response = generate_response(chat, model="llama-3.1-8b-instant", max_tokens=1000, temperature=0.8)
chat = add_ai_message(chat, response)

# Print the output
print("\nOne-Shot Story Generation Output:\n")
print(response)


One-Shot Story Generation Output:

**Brainstorm**


Here are three unique story ideas in different genres:


1. **The Last Dreamwalker** (Fantasy): In a world where dreams are a parallel reality, a young woman must master the ancient art of dreamwalking to
prevent a catastrophic dream invasion that threatens to destroy the boundaries between the waking and dream worlds.

2. **The Memory Thief** (Science Fiction): When a talented memory thief is hired to steal a powerful CEO's memories, he discovers a dark conspiracy
that threatens the very fabric of society, forcing him to use his skills to uncover the truth and protect his own identity.

3. **The Lost City of Echoes** (Historical Fiction): In 1920s Egypt, a brilliant archaeologist uncovers a hidden city deep within the desert, only to
discover that its secrets are shrouded in a mysterious curse that has haunted the city for centuries.


**Explore**


I choose to explore the first idea, **The Last Dreamwalker**. Here's a brief descrip

### Chains

In [12]:
# Initialize the conversation history
chat = []
chat = add_system_message(chat, "You are a helpful and creative assistant who excels at story writing.")

# Step 1: Brainstorm story ideas
step_1_prompt = (
    "Let's brainstorm story ideas. Please generate three interesting and unique story ideas "
    "in different genres. Be creative and ensure each idea is summarized in a single sentence."
)
chat = add_user_message(chat, step_1_prompt)
response = generate_response(chat, model="llama-3.1-8b-instant", max_tokens=200, temperature=0.9)
chat = add_ai_message(chat, response)
print("\nStep 1 - Story Ideas:\n")
print(response)

# Parse response and choose one idea to explore further

chosen_idea = input("Which of the three story ideas would you like to explore further? ")

# Step 2: Explore the chosen story idea
step_2_prompt = (
    f"I like {chosen_idea}. Can you expand on this idea? Describe the main plot points, the protagonist, "
    "and any other key details that will make this story interesting."
)
chat = add_user_message(chat, step_2_prompt)
response = generate_response(chat, model="llama-3.1-8b-instant", max_tokens=300, temperature=0.7)
chat = add_ai_message(chat, response)
print("\nStep 2 - Expanded Story Idea:\n")
print(response)

# Step 3: Generate a detailed outline for the story
step_3_prompt = (
    "Now that we have expanded the idea, can you create a detailed outline for this story? "
    "Include the main structure: beginning, middle, and end, as well as key events or twists."
)
chat = add_user_message(chat, step_3_prompt)
response = generate_response(chat, model="llama-3.1-8b-instant", max_tokens=350, temperature=0.7)
chat = add_ai_message(chat, response)
print("\nStep 3 - Story Outline:\n")
print(response)

# Step 4: Refine and summarize
step_4_prompt = (
    "Based on the outline, please summarize this story in a concise and engaging paragraph "
    "suitable for a pitch or a back-cover description."
)
chat = add_user_message(chat, step_4_prompt)
response = generate_response(chat, model="llama-3.1-8b-instant", max_tokens=150, temperature=0.5)
chat = add_ai_message(chat, response)
print("\nStep 4 - Final Summary:\n")
print(response)


Step 1 - Story Ideas:

Here are three unique story ideas across different genres:


**Genre: Science Fantasy**

In "The Last Symphony," a young musician discovers she has the ability to manipulate the fabric of reality through her music, and she must use her
newfound powers to repair the damaged cosmos and restore harmony to a world on the brink of collapse.


**Genre: Psychological Thriller**

"Memory Thief" follows a detective who, after being diagnosed with a rare neurological condition that steals his memories, must navigate a complex web
of deceit and conspiracy to uncover the truth about his own past, even as his memories – and his grip on reality – continue to slip away.


**Genre: Utopian Dystopia**

Set in a world where emotions have been genetically engineered out of existence, "The Muted Dawn" tells the story of a young outcast who discovers a
hidden oasis of feeling and passion within herself, and must decide whether to join the rebellion that seeks to overthrow the emotio

### Try it!

Think of problem or situation that might require multiple steps to solve or a linear process that could benefit from feedback at various stages. Work with a partner to build out a chain for it.

##### Some Examples (or pick your own):

Plan a birthday party. Possible workflow:
- Step 1: Brainstorm a list of possible themes for the party.
- Step 2: Refine the list by selecting the most practical or appealing theme based on feedback.
- Step 3: Generate a list of activities and decorations tailored to the chosen theme.
- Step 4: Create a detailed schedule or shopping list based on the plan.


Diagnosing or troubleshoot a tech issue. Possible workflow:
- Step 1: Gather information about the problem (e.g., symptoms, error messages, device details).
- Step 2: Suggest a set of potential solutions or tests to narrow down the issue.
- Step 3: Incorporate feedback from tests and refine the next steps.
- Step 4: Provide a final resolution or guide the user to seek expert help.

In [13]:
# TODO: Your code here